# Sesión 2 — Preparación y limpieza de datos con pandas (Telecom)

"
"Notebook para ejecutar en directo durante la sesión.

"
"## Objetivo
Dejar un dataset **entrenable** y **reproducible**.

"
"## Entregable
- `tickets_telecom_clean.csv`
- `decisiones.txt` (5 líneas)


In [2]:
# Imports
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 60)
pd.set_option('display.width', 140)


## 0) Dataset sintético (telecom)
Si no tienes datos reales para la demo, genera un dataset parecido a tickets/SLA.


In [3]:
# Generador de dataset sintético (telecom)
rng = np.random.default_rng(42)
N = 1200

zonas = ['Granada Norte','Granada Sur','Alhendín','Motril','Baza']
canales = ['APP','LLAMADA','EMAIL']
tipos = ['FTTH down','4G/5G sin servicio','Latencia alta','Cortes intermitentes','Router reinicios']
vip = ['SÍ','NO']

base = pd.Timestamp('2025-01-01')
fechas = base + pd.to_timedelta(rng.integers(0, 50, size=N), unit='D')

kpi_lat = np.clip(rng.normal(40, 25, size=N), 0, 400)
kpi_jit = np.clip(rng.normal(8, 6, size=N), 0, 120)

# Texto / descripción
frases = [
  'sin servicio desde esta mañana',
  'cortes intermitentes por la tarde',
  'latencia alta en videollamada',
  'router reinicia solo',
  'sin conexión en toda la zona',
  'microcortes cada 10 minutos'
]
descripcion = rng.choice(frases, size=N)

# Prioridad (target) - heurística telecom
prio = []
for i in range(N):
    score = 0
    if 'sin servicio' in descripcion[i]: score += 2
    if 'sin conexión' in descripcion[i]: score += 2
    if kpi_lat[i] > 90: score += 2
    if kpi_jit[i] > 25: score += 1
    if rng.choice(vip) == 'SÍ': score += 1
    prio.append('ALTA' if score >= 4 else ('MEDIA' if score >= 2 else 'BAJA'))

# Dataset
df = pd.DataFrame({
    'id_ticket': np.arange(1, N+1),
    'fecha_creación': fechas,
    'zona': rng.choice(zonas, size=N),
    'canal': rng.choice(canales, size=N),
    'tipo_incidencia': rng.choice(tipos, size=N),
    'cliente_vip': rng.choice(vip, size=N, p=[0.12, 0.88]),
    'kpi_lat': kpi_lat,
    'kpi_jit': kpi_jit,
    'descripcion': descripcion,
    'prioridad': prio,
})

# Inyectar nulos y ruido
for col, p in [('kpi_lat',0.10), ('kpi_jit',0.06), ('zona',0.02)]:
    mask = rng.random(N) < p
    df.loc[mask, col] = np.nan

# Variantes en categorías
variants = {'Granada Norte': ['GRANADA NORTE','Granada Norte ', 'granada norte']}
mask = (df['zona'] == 'Granada Norte') & (rng.random(N) < 0.12)
df.loc[mask, 'zona'] = rng.choice(variants['Granada Norte'], size=mask.sum())

# Duplicados simulados
if N > 50:
    dup_rows = df.sample(10, random_state=1)
    df = pd.concat([df, dup_rows], ignore_index=True)

df.head()

,id_ticket,fecha_creación,zona,canal,tipo_incidencia,cliente_vip,kpi_lat,kpi_jit,descripcion,prioridad
0,1,2025-01-05,Granada Sur,APP,Latencia alta,SÍ,71.198750,9.264001,cortes intermitentes por la tarde,BAJA
1,2,2025-02-08,Granada Norte,LLAMADA,4G/5G sin servicio,NO,33.687067,1.271414,router reinicia solo,BAJA
2,3,2025-02-02,Baza,LLAMADA,Cortes intermitentes,NO,49.086358,16.774132,sin conexión en toda la zona,MEDIA
3,4,2025-01-22,Granada Norte,APP,Latencia alta,NO,0.000000,12.855168,microcortes cada 10 minutos,BAJA
4,5,2025-01-22,Motril,EMAIL,Cortes intermitentes,NO,11.091308,8.572634,sin conexión en toda la zona,MEDIA


## 1) Carga y exploración (EDA)
- Nulos
- Duplicados
- Categorías
- Desbalanceo del target


In [4]:
missing = df.isna().mean().sort_values(ascending=False)
print('Top nulos:')
print(missing.head(10))

print('\nDuplicados por id_ticket:', df.duplicated('id_ticket').mean())
print('\nTipo incidencia top:')
print(df['tipo_incidencia'].value_counts().head(5))
print('\nZonas top:')
print(df['zona'].astype('string').value_counts().head(5))
print('\nPrioridad (desbalanceo):')
print(df['prioridad'].value_counts(normalize=True))


Top nulos:
kpi_lat            0.091736
kpi_jit            0.061157
zona               0.028926
id_ticket          0.000000
fecha_creación     0.000000
canal              0.000000
cliente_vip        0.000000
tipo_incidencia    0.000000
descripcion        0.000000
prioridad          0.000000
dtype: float64

Duplicados por id_ticket: 0.008264462809917356

Tipo incidencia top:
tipo_incidencia
Cortes intermitentes    266
Router reinicios        243
Latencia alta           240
FTTH down               240
4G/5G sin servicio      221
Name: count, dtype: int64

Zonas top:
zona
Alhendín         255
Baza             236
Motril           234
Granada Sur      226
Granada Norte    202
Name: count, dtype: Int64

Prioridad (desbalanceo):
prioridad
BAJA     0.632231
MEDIA    0.359504
ALTA     0.008264
Name: proportion, dtype: float64


### Ejercicio 1 (6–8 min)
1) Top 5 nulos
2) p99 de kpi_lat y kpi_jit
3) % duplicados por id_ticket
4) 3 decisiones de limpieza justificadas


In [5]:
# 1) Top 5 nulos
missing = df.isna().mean().sort_values(ascending=False)
print('Top 5 nulos:')
print(missing.head(5))

# 2) p99 de kpi_lat y kpi_jit
print('\np99 de kpi_lat:', df['kpi_lat'].quantile(0.99))
print('p99 de kpi_jit:', df['kpi_jit'].quantile(0.99))

# 3) % duplicados por id_ticket
print('\n% duplicados por id_ticket:', df.duplicated('id_ticket').mean() * 100)

# 4) 3 decisiones de limpieza justificadas
print('\nDecisiones de limpieza:')
print('1) Imputar los valores nulos en kpi_lat y kpi_jit con la mediana, ya que son métricas numéricas y la mediana es robusta a outliers.')
print('2) Eliminar duplicados basados en id_ticket, ya que cada ticket debe ser único.')
print('3) Estandarizar las categorías en la columna zona, convirtiendo variantes como "GRANADA NORTE" a "Granada Norte" para consistencia.')

Top 5 nulos:
kpi_lat           0.091736
kpi_jit           0.061157
zona              0.028926
id_ticket         0.000000
fecha_creación    0.000000
dtype: float64

p99 de kpi_lat: 99.3691616169896
p99 de kpi_jit: 22.24091397003272

% duplicados por id_ticket: 0.8264462809917356

Decisiones de limpieza:
1) Imputar los valores nulos en kpi_lat y kpi_jit con la mediana, ya que son métricas numéricas y la mediana es robusta a outliers.
2) Eliminar duplicados basados en id_ticket, ya que cada ticket debe ser único.
3) Estandarizar las categorías en la columna zona, convirtiendo variantes como "GRANADA NORTE" a "Granada Norte" para consistencia.


## 2) Limpieza mínima viable (MVP)
- Política de nulos (imputar + flags)
- Normalizar categorías
- Texto: texto_norm + len_texto + keywords


In [8]:
# Nulos numéricos (ejemplo) + flags
for col in ['kpi_lat','kpi_jit']:
    df[f'{col}_disponible'] = df[col].notna().astype(int)
    df[f'{col}_imputado'] = df[col].isna().astype(int)
    df[col] = df[col].fillna(df[col].median())

# Categorías
df['zona'] = df['zona'].astype('string').str.strip().str.title().fillna('Sin info')
df['canal'] = df['canal'].astype('string').str.strip().str.upper().fillna('Sin info')
df['cliente_vip'] = df['cliente_vip'].astype('string').str.strip().str.upper().replace({'SI':'SÍ','YES':'SÍ'}).fillna('NO')

# Texto
txt = df['descripcion'].fillna('')
df['texto_norm'] = (txt.str.lower().str.replace(r'\s+',' ', regex=True).str.strip())
df['len_texto'] = df['texto_norm'].str.len()
df['kw_sin_servicio'] = df['texto_norm'].str.contains('sin servicio|sin conexion|caid', regex=True).astype(int)

df.head()

,id_ticket,fecha_creación,zona,canal,tipo_incidencia,cliente_vip,kpi_lat,kpi_jit,descripcion,prioridad,kpi_lat_disponible,kpi_lat_imputado,kpi_jit_disponible,kpi_jit_imputado,texto_norm,len_texto,kw_sin_servicio
0,1,2025-01-05,Granada Sur,APP,Latencia alta,SÍ,71.198750,9.264001,cortes intermitentes por la tarde,BAJA,1,0,1,0,cortes intermitentes por la tarde,33,0
1,2,2025-02-08,Granada Norte,LLAMADA,4G/5G sin servicio,NO,33.687067,1.271414,router reinicia solo,BAJA,1,0,1,0,router reinicia solo,20,0
2,3,2025-02-02,Baza,LLAMADA,Cortes intermitentes,NO,49.086358,16.774132,sin conexión en toda la zona,MEDIA,1,0,1,0,sin conexión en toda la zona,28,0
3,4,2025-01-22,Granada Norte,APP,Latencia alta,NO,0.000000,12.855168,microcortes cada 10 minutos,BAJA,1,0,1,0,microcortes cada 10 minutos,27,0
4,5,2025-01-22,Motril,EMAIL,Cortes intermitentes,NO,11.091308,8.572634,sin conexión en toda la zona,MEDIA,1,0,1,0,sin conexión en toda la zona,28,0


### Ejercicio 2 (8–12 min)
- Define 3 reglas de limpieza (1 numérica, 1 categórica, 1 texto)
- Escribe 5 líneas con tus decisiones


In [ ]:
# Espacio del alumno
# Decisiones (5 líneas):
# 1) Eliminar valores atípicos en la columna 'edad' utilizando el rango intercuartílico (IQR).
# 2) Rellenar valores faltantes en la columna 'género' con la moda.
# 3) Eliminar espacios en blanco al inicio y final de las cadenas en la columna 'nombre'.
# 4) Convertir todas las cadenas en la columna 'categoría' a minúsculas para estandarización.
# 5) Sustituir valores nulos en la columna 'ingresos' con el promedio de la columna.

## 3) Preparación para ML (sin entrenar)
- One‑Hot de categóricas
- Escalado de numéricos
- ColumnTransformer


In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

X = df.drop(columns=['prioridad'])
y = df['prioridad']

cat_cols = ['canal','zona','tipo_incidencia','cliente_vip']
num_cols = ['kpi_lat','kpi_jit','len_texto','kw_sin_servicio']

pre = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols),
    ('num', StandardScaler(), num_cols),
])

X_pre = pre.fit_transform(X)
print('X_pre shape:', X_pre.shape)

X_pre shape: (1210, 20)


### Ejercicio 3 (10–12 min)
- Ajusta cat_cols/num_cols
- Explica si el número de columnas crece demasiado y por qué


In [10]:
# Espacio del alumno

# Ajustar cat_cols y num_cols
cat_cols = ['zona', 'tipo_incidencia', 'cliente_vip']
num_cols = ['kpi_lat', 'kpi_jit', 'len_texto']

# Explicación sobre el crecimiento de columnas
# El número de columnas puede crecer significativamente debido al One-Hot Encoding.
# Cada categoría única en las columnas categóricas se convierte en una nueva columna.
# Por ejemplo, si 'zona' tiene 5 categorías, se crearán 5 columnas adicionales.
# Esto puede llevar a un aumento considerable en la dimensionalidad del conjunto de datos,
# especialmente si hay muchas categorías únicas en las columnas categóricas.

## 4) Split correcto + anti‑leakage
- Split temporal
- Evitar columnas del cierre


In [11]:
df = df.sort_values('fecha_creación')
cut = int(len(df)*0.8)
train_df = df.iloc[:cut]
test_df  = df.iloc[cut:]

print('train:', train_df.shape, 'test:', test_df.shape)

train: (968, 17) test: (242, 17)


### Ejercicio 4 (6–8 min)
Marca como PROHIBIDA si se conoce después de decidir prioridad:
- fecha_cierre
- tiempo_resolución_h
- técnico_asignado (si existiera)

Criterio: “¿estaba disponible al crear el ticket?”


In [ ]:
# Espacio del alumno
# a) fecha_cierre, b) canal, c) tiempo_resolucion, d) kpi_lat, e) tecnico_asignado, f) descripcion

## 5) Guardar entregable
- Dataset limpio
- Resumen (5 líneas)


In [ ]:
df.to_csv('tickets_telecom_clean.csv', index=False)

resumen = """1) Normalizamos zona/canal/tipo (strip + title).
2) Imputamos kpi_lat y kpi_jit con mediana + flags.
3) Creamos texto_norm, len_texto y kw_sin_servicio.
4) Excluimos campos del cierre para evitar leakage.
5) Split temporal 80/20 para simular producción."""

with open('decisiones.txt','w', encoding='utf-8') as f:
    f.write(resumen)

print('Generado: tickets_telecom_clean.csv + decisiones.txt')

Generado: tickets_telecom_clean.csv + decisiones.txt
